In [26]:
from src.video_processing import *
import pandas as pd

gpu=True

def deteccao_com_processamento(video_path):

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erro ao abrir o vídeo: {video_path}")

    leitor = easyocr.Reader(['pt', 'en'], gpu=gpu, verbose=False)

    # Placa atual inferida
    placa_atual = None  
    # Lista de contadores de votos por posição de caractere
    votos_caracteres = []

    todas_as_placas = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resultado = inferencia(frame)
        if len(resultado.boxes) > 0:  # Verificar se há alguma placa detectada
            bbox = resultado.boxes.xyxy[0].tolist()

            placa_img = cortar_placa(frame, bbox)

            texto_detectado = detectar_caracteres(placa_img, leitor)
            if texto_detectado is not None:
                if placa_atual is None:
                    # Primeira placa detectada
                    placa_atual = texto_detectado
                    votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                else:
                    if detectar_nova_placa(placa_atual, texto_detectado):
                        # Considerar uma nova placa
                        placa_atual = texto_detectado
                        votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                    else:
                        # Atualizar os votos de cada caractere
                        atualizar_votacao_caracteres(placa_atual, texto_detectado, votos_caracteres)
                        # Formar a nova placa inferida com base nos votos
                        placa_inferida = formar_placa_por_votacao(votos_caracteres)
                        if placa_inferida != placa_atual:
                            placa_atual = placa_inferida

                todas_as_placas.append(placa_atual)
            
    cap.release()

    return todas_as_placas

def deteccao_sem_processamento(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erro ao abrir o vídeo: {video_path}")

    leitor = easyocr.Reader(['pt', 'en'], gpu=gpu, verbose=False)

    todas_as_placas_sem_proc = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resultado = inferencia(frame)
        if len(resultado.boxes) > 0:  # Verificar se há alguma placa detectada
            bbox = resultado.boxes.xyxy[0].tolist()

            placa_img = cortar_placa(frame, bbox)

            resultado = leitor.readtext(np.array(placa_img))

            texto_detectado = " ".join([res[1] for res in resultado])
            
            # Verificar se a placa tem 7 caracteres
            if len(texto_detectado) == 7:
                todas_as_placas_sem_proc.append(texto_detectado)

    cap.release()

    return todas_as_placas_sem_proc

In [27]:
video_paths = [
    "organic-data/video_teste_1_compare.mp4",
    "organic-data/video_teste_2_compare.mp4",
    "organic-data/video_teste_3_compare.mp4"
]

video_placas = [
    "MVJ0317",
    "RGF5A86",
    "SLA3G28"
]

resultado = {}

for placa, path in zip(video_placas, video_paths):
    com_proc = deteccao_com_processamento(path)
    
    sem_proc = deteccao_sem_processamento(path)
    
    resultado[placa] = {
        "com_processamento": com_proc,
        "sem_processamento": sem_proc
    }


0: 640x384 1 object, 4.8ms
Speed: 2.2ms preprocess, 4.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 5.3ms
Speed: 1.8ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 5.3ms
Speed: 2.0ms preprocess, 5.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 4.2ms
Speed: 1.6ms preprocess, 4.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 4.9ms
Speed: 1.4ms preprocess, 4.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 6.5ms
Speed: 2.6ms preprocess, 6.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 4.2ms
Speed: 1.3ms preprocess, 4.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x

In [40]:
estatisticas = {}

for placa in video_placas:

    estatisticas[placa] = {
        "com_processamento_media_de_acertos": np.mean(np.array(resultado[placa]["com_processamento"]) == placa),
        "sem_processamento_media_de_acertos": np.mean(np.array(resultado[placa]["sem_processamento"]) == placa)
    }

In [42]:
pd.DataFrame(estatisticas).T

,com_processamento_media_de_acertos,sem_processamento_media_de_acertos
MVJ0317,0.888889,0.285714
RGF5A86,1.000000,0.680000
SLA3G28,0.867925,0.274510


In [44]:
print(pd.DataFrame(estatisticas).T.to_latex())

\begin{tabular}{lrr}
\toprule
 & com_processamento_media_de_acertos & sem_processamento_media_de_acertos \\
\midrule
MVJ0317 & 0.888889 & 0.285714 \\
RGF5A86 & 1.000000 & 0.680000 \\
SLA3G28 & 0.867925 & 0.274510 \\
\bottomrule
\end{tabular}

